# Suffix Tree Term

#### Made by Yuliia Hetman

In [5]:
from collections import defaultdict
from typing import Optional


class Node:
    text = ""  # for debug purpose

    def __init__(self, edges=None, suff_link=None, parent=None):
        self.edges = edges if edges is not None else defaultdict(type(None))
        self.suff_link: Optional[Node] = suff_link
        self.parent: Optional[Node] = parent

    def __str__(self):
        if self.parent:
            idxs = [e.idxs for e in self.parent.edges.values() if e.target_node == self][0]
            return f"{self.parent}{self.text[idxs[0]: idxs[1]]}"
        else:
            return "$"


class Edge:
    def __init__(self, idxs=None, target_node=None):
        self.idxs = idxs
        self.target_node: Optional[Node] = target_node


class SuffixTree:
    # node =  < suffix_link, parent, edges >
    # edges = { first_char => edge }
    # edge =  < idxs, target_node >
    # idxs = (first, last) # corresponds to label = text[first: last]
    def __init__(self, text):
        self.text = text
        self.joker_edge = Edge(idxs=(0, 1))
        self.joker = Node(edges=defaultdict(lambda: self.joker_edge))
        self.root = Node(suff_link=self.joker)
        self.joker_edge.target_node = self.root
        self.infty = len(self.text)
        self._build_tree(self.root, 0, self.infty)

    def __str__(self):
        return f"SuffixTree(text='{self.text}')"

    def pp(self, node=None, indent=0):
        node = node or self.root
        space = "    " * indent
        print(space + f"ID    : {node}")
        print(space + f"link  : {node.suff_link}")
        print(space + f"edges : ")

        for c, edge in node.edges.items():
            print(space + f"  -{c} {edge.idxs}={self.text[edge.idxs[0]: edge.idxs[1]]}:")
            self.pp(edge.target_node, indent + 1)

    def _build_tree(self, node: Node, n: int, infty: int, skip: int = 0):
        while n < infty:
            c = self.text[n]
            edge = node.edges[c]
            if edge is not None:
                first, last = edge.idxs
                i, n0 = first, n
                if skip > 0:
                    can_skip = min(skip, last - first)
                    i += can_skip
                    n += can_skip
                    skip -= can_skip

                while (
                    i < last and n < infty and
                    (self.text[i] == self.text[n] or edge is self.joker_edge)
                ):
                    i += 1
                    n += 1

                if i == last:  # go to the next node
                    node = edge.target_node
                else:  # splitting edge
                    middle_node = Node(parent=node)
                    middle_node.edges[self.text[i]] = edge
                    node.edges[c] = Edge(idxs=(first, i), target_node=middle_node)
                    edge.idxs = (i, edge.idxs[1])
                    edge.target_node.parent = middle_node
                    middle_node.suff_link = self._build_tree(node.suff_link, n0, n, i - first)
                    node = middle_node
            else:  # no way to go; creating new leaf
                new_leaf = Node(parent=node)
                node.edges[c] = Edge(idxs=(n, self.infty), target_node=new_leaf)
                node = node.suff_link
        return node


In [ ]:
%%time
from multiprocessing import Pool
import re, os, json
from string import digits
import pandas as pd

docIDs = list(range(0, 18))

def delete_sings_digits(content):
    punc = '''!()-'”№[]{};:"\,“«»<>./?@#$%—…^&*_~|–abcdefghijklmnoqprstuvwxyz'''
    content = "".join([ele for ele in content.lower() if ele not in punc])
    table = str.maketrans('', '', digits)
    content = content.translate(table)
    return content


def create_vocabulary(content):    
    raw_content = delete_sings_digits(content)
    vocab = [i.lower() for i in raw_content.split() if i != '']
    return vocab

def read_book(book):
    dirname='../books/'
#     tree = Tree()
    with open(dirname + book, 'r', encoding='utf-8') as file:
        text = create_vocabulary(file.read())
#     unique = list(set(text))
#     for key in unique:
#         tree.insert(key)
    return text


def isUkrainian(word):
    ukrainian = [96, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1100, 1102, 1103, 1108, 1110, 1111, 1169, 8217]
    for l in word:
        if ord(l) not in ukrainian:
#             print("NOT UKRAINIAN LETTER:", l, "IN WORD", key)
            return False
    return True
       
        
def get_vocabulary(dirname='../books/'):
    books = os.listdir(dirname)
    vocabulary = list()
    dictionary = dict()
    memory = 0
    with Pool(8) as p:
        vocabul = p.map(read_book, books)
    for i, book in enumerate(books):
        dictionary[book.replace('.txt', '')] = vocabul[i]
        vocabulary += vocabul[i]
        print(book, "is read!")
        memory += os.stat(dirname + book).st_size
    return memory, vocabulary, dictionary


memory, vocabulary, dictionary = get_vocabulary()
print(f'Total number of words : {len(vocabulary)}')
print(f'Total number of tokens : {len(set(vocabulary))}')


In [6]:

ukr_voc = list()

for key in vocabulary:
    if isUkrainian(key) == True:
        ukr_voc.append(key)
        Node.text = key
        tree = SuffixTree(key)
tree.pp()

ID    : $
link  : $
edges : 
  -I (0, 12)=I am Yuliia.:
    ID    : $I am Yuliia.
    link  : None
    edges : 
  -  (1, 2)= :
    ID    : $ 
    link  : $
    edges : 
      -a (2, 12)=am Yuliia.:
        ID    : $ am Yuliia.
        link  : None
        edges : 
      -Y (5, 12)=Yuliia.:
        ID    : $ Yuliia.
        link  : None
        edges : 
  -a (2, 3)=a:
    ID    : $a
    link  : $
    edges : 
      -m (3, 12)=m Yuliia.:
        ID    : $am Yuliia.
        link  : None
        edges : 
      -. (11, 12)=.:
        ID    : $a.
        link  : None
        edges : 
  -m (3, 12)=m Yuliia.:
    ID    : $m Yuliia.
    link  : None
    edges : 
  -Y (5, 12)=Yuliia.:
    ID    : $Yuliia.
    link  : None
    edges : 
  -u (6, 12)=uliia.:
    ID    : $uliia.
    link  : None
    edges : 
  -l (7, 12)=liia.:
    ID    : $liia.
    link  : None
    edges : 
  -i (8, 9)=i:
    ID    : $i
    link  : $
    edges : 
      -i (9, 12)=ia.:
        ID    : $iia.
        link  : None
   